In [3]:
df=pd.read_csv('data_agg_final.csv', index_col=0)

In [2]:
import pandas as pd

In [4]:
df

,Открыт_дата,День недели,Месяц,Год,Неделя,ЗП,Время года,"Сумма по чеку, руб.",День,Средняя Температура,Снег,Дождь,Праздник_вых,Праздник_раб,Праздники,Предпраздничный,Выходной
0,2022-09-01,3,9,2022,1,0,3,96630.00,1,11.4,0,0,False,1,1,0.0,0
1,2022-09-02,4,9,2022,1,0,3,108024.50,2,10.5,0,1,False,0,0,0.0,0
2,2022-09-03,5,9,2022,1,0,3,122352.00,3,12.5,0,0,False,0,0,0.0,1
3,2022-09-04,6,9,2022,1,0,3,119441.70,4,12.2,0,0,False,0,0,0.0,1
4,2022-09-05,0,9,2022,1,0,3,87863.50,5,10.3,0,0,False,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,2023-10-22,6,10,2023,4,0,3,138996.12,22,2.6,1,0,False,0,0,0.0,1
416,2023-10-23,0,10,2023,4,0,3,61260.00,23,4.4,0,1,False,0,0,0.0,0
417,2023-10-24,1,10,2023,4,0,3,70993.00,24,1.4,0,1,False,0,0,0.0,0
418,2023-10-25,2,10,2023,4,0,3,85317.97,25,-0.3,0,0,False,0,0,0.0,0


In [121]:
import numpy as np

In [122]:
df['Выходной'] = df['День недели'].apply(lambda x: 1 if x in (5,6) else 0)

In [125]:
df = df.drop(columns=['Предпраздничный'])

In [126]:
df['Праздник_раб'] = np.where((pd.to_datetime(df['Открыт_дата']).dt.month==1) 
                              & (pd.to_datetime(df['Открыт_дата']).dt.day==25), 1, df['Праздник_раб'])

In [127]:
df['Праздник_раб'] = np.where((pd.to_datetime(df['Открыт_дата']).dt.month==2) 
                              & (pd.to_datetime(df['Открыт_дата']).dt.day==14), 1, df['Праздник_раб'])

In [128]:
df['Праздник_раб'] = np.where((pd.to_datetime(df['Открыт_дата']).dt.month==9) 
                              & (pd.to_datetime(df['Открыт_дата']).dt.day==1), 1, df['Праздник_раб'])

In [129]:
df['Праздник_раб'] = np.where((pd.to_datetime(df['Открыт_дата']).dt.month==10) 
                              & (pd.to_datetime(df['Открыт_дата']).dt.day==5), 1, df['Праздник_раб'])

In [130]:
def easter_date(god):
    a = god % 19
    b = god % 4
    c = god % 7
    d = (19 * a + 15) % 30
    e = (2 * b + 4 * c + 6 * d + 6) % 7
    f = d + e
    if f <= 26:
        rez = '04-' + str(f + 4) 
    else:
        rez = + '05-' + str(f - 26) 
    return str(god) + '-' + rez

In [131]:
df['Праздник_раб'] = df.apply(lambda row: 1 
if easter_date(row['Год']) == row['Открыт_дата']
                             else row['Праздник_раб'], axis=1)

In [132]:
df['Праздники'] = df['Праздник_вых'].astype(int) + df['Праздник_раб']

In [118]:
df['Предпраздничный'] = df['Праздники'].shift(-1)
mask = df['Предпраздничный'] == df['Праздники']

# Проставляем ноль во второй колонке там, где значения не равны
df.loc[mask, 'Предпраздничный'] = 0

# Заменяем NaN в последней строке значением False
df.at[df.index[-1], 'Предпраздничный'] = 0

In [133]:
df.to_csv('data_raw_final.csv')